# Amazon Textract LangChain Document Loader

Amazon Textract is a machine learning (ML) service that automatically extracts text, handwriting, and data from scanned documents. It goes beyond simple optical character recognition (OCR) to identify, understand, and extract data from forms and tables. Today, many companies manually extract data from scanned documents such as PDFs, images, tables, and forms, or through simple OCR software that requires manual configuration (which often must be updated when the form changes). To overcome these manual and expensive processes, Textract uses ML to read and process any type of document, accurately extracting text, handwriting, tables, and other data with no manual effort. You can quickly automate document processing and act on the information extracted, whether you’re automating loans processing or extracting information from invoices and receipts. Textract can extract the data in minutes instead of hours or days.

This sample demonstrates the use of Amazon Textract in combination with LangChain as a DocumentLoader.

Textract supports PDF, TIFF, PNG and JPEG format.

Check https://docs.aws.amazon.com/textract/latest/dg/limits-document.html for supported document sizes, languages and characters.

In [ ]:
%pip install -q langchain boto3 openai tiktoken python-dotenv

## Sample 1

The first example uses a local file, which internally will be send to Amazon Textract sync API [DetectDocumentText](https://docs.aws.amazon.com/textract/latest/dg/API_DetectDocumentText.html). 

Local files or URL endpoints like HTTP:// are limited to one page documents for Textract.
Multi-page documents have to reside on S3.

In [ ]:
from langchain.document_loaders import AmazonTextractPDFLoader
loader = AmazonTextractPDFLoader("example_data/w2-example.pdf")
documents = loader.load()

Output from the file

In [ ]:
documents

## Sample 2
The next sample loads a file from an HTTPS endpoint. 
It has to be single page, as Amazon Textract requires all multi-page documents to be stored on S3.

In [ ]:
loader = AmazonTextractPDFLoader("https://amazon-textract-public-content.s3.us-east-2.amazonaws.com/langchain/alejandro_rosalez_sample_1.jpg")
documents = loader.load()

In [ ]:
documents

## Sample 3

Process a multi-page document from S3. The sample document resides in a bucket in us-east-2 and Textract needs to be called in that same region to be successful.
So for this sample to work, you have to have your SageMaker Notebook running in us-east-2 or when running in a different environment, pass in a boto3 Textract client with that region name.

In [ ]:
import boto3
textract_client = boto3.client('textract', region_name='us-east-2')

file_path = "s3://amazon-textract-public-content/langchain/layout-parser-paper.pdf"
loader = AmazonTextractPDFLoader(file_path, client=textract_client)
documents = loader.load()

Get the number of pages to validate the response (printing out the full response would be quite long...). We expect 16 pages.

In [ ]:
len(documents)

## Using the AmazonTextractPDFLoader in an LangChain chain (e. g. OpenAI)

The AmazonTextractPDFLoader can be used in a chain the same way the other loaders are used.
Textract itself does have a [Query feature](https://docs.aws.amazon.com/textract/latest/dg/API_Query.html), which offers similar functionality to the QA chain in this sample, which is worth checking out as well.

In [ ]:
# You can store your OPENAI_API_KEY in a .env file as well
# import os 
# from dotenv import load_dotenv

# load_dotenv()

In [ ]:
# Or set the OpenAI key in the environment directly
import os 
os.environ["OPENAI_API_KEY"] = "your-OpenAI-API-key"

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm=OpenAI(), chain_type="map_reduce")
query = ["Who are the autors?"]

chain.run(input_documents=documents, question=query)